# Batch Testing Tutorial

This tutorial has two roles:

1. Be familiar with our code.
2. Reproduce the results



In [5]:
import numpy as np
import pandas as pd
from numba import jit
import time
import numba
import fast_btk as fbtk
from sklearn.metrics import precision_score, recall_score
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data generation

The function `data_gen` can generate a population with a certain size $N$ and infection rate $p$

In [2]:
fbtk.data_gen(size = 10, p = 0.1)

array([[0, 0],
       [1, 0],
       [2, 0],
       [3, 0],
       [4, 0],
       [5, 0],
       [6, 0],
       [7, 0],
       [8, 0],
       [9, 0]])

# Conventional Test

`conventional_test` gives the test results to a subject array given the probability of a type II error, the probability of a type I error, the number of repeatition, and setting of sequence testing or not.


In [3]:
subject_array = fbtk.data_gen(10, 0.1)
test_result, consum = fbtk.conventional_test(subject_array, typeII_error = 0.15,
typeI_error=0.01, repeat= 1)
print(f'accuracy: {np.mean(subject_array[:,1] == test_result[:,1])}')
print(f'test consumption {consum}')

accuracy: 0.9
test consumption 10


## Multi-step Batch Testing

`seq_test` gives the test results to a subject array, the total number of 
test-kit consumption, and the number of individual tests given the subject array,
stopping rule, batch size, probability of a type II error, probability of a Type I error, and the number of repeatition, probability threshold, and 
setting of sequence testing or not.

The following code will generate a population with size 100000 and the infection rate of 0.01. The setting of this multi-step batch testing is up to 3 sequential individual tests for 3 batch positives.

In [4]:
subject_array = fbtk.data_gen(100000, 0.01)
batch_size = fbtk.one_batch_test_int_solver(0.01, 0.15, 0.01)
test_result, consum, ind_consum = fbtk.seq_test(subject_array, batch_size = batch_size,stop_rule = 3,p = 0.01, typeII_error = 0.15, typeI_error=0.01, repeat= 3, seq = True)
print(f'accuracy: {np.mean(subject_array[:,1] == test_result[:,1])}')
print(f'test consumption {consum}')

accuracy: 0.99891
test consumption 28203.0


# Reproduce Results

The following code is to produce results on Table 7 and Table 8. We will go through table 7_(a) and show the output. 

In [6]:
# table 7 (a)
time_start = time.time()
np.random.seed(0)
df = pd.DataFrame([], columns = ['Infection_rate', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD'])
for prob in [0.001, 0.01, 0.03, 0.05, 0.10]:
    temp_data = [fbtk.data_gen(100000, prob) for _ in range(100)]
    length = len(temp_data)
    acc = np.zeros(length)
    sens = np.zeros(length)
    spec = np.zeros(length)
    ppv = np.zeros(length)
    npv = np.zeros(length)
    test_consum = np.zeros(length)
    for i in range(length):
        pred, consum = fbtk.conventional_test(temp_data[i], typeII_error= 0.15, typeI_error=0.01)
        acc[i] = np.mean(pred[:,1] == temp_data[i][:, 1])
        sens[i] = recall_score(temp_data[i][:, 1], pred[:, 1])
        spec[i] = fbtk.specificity_score(temp_data[i][:, 1], pred[:, 1])
        ppv[i] = precision_score(temp_data[i][:, 1], pred[:, 1])
        npv[i] = fbtk.npv_score(temp_data[i][:, 1], pred[:, 1])
        test_consum[i] = consum
    result = {
        'acc': acc,
        'sens': sens,
        'spec': spec,
        'PPV': ppv,
        'NPV': npv,
        'test_consum': test_consum
    
    }
    result = pd.DataFrame(result)
    result_mean = result.mean()
    result_std = result.std()
    temp_df = [prob, result_mean['acc'], result_std['acc'], result_mean['sens'], result_std['sens'],
    result_mean['spec'], result_std['spec'], result_mean['PPV'], result_std['PPV'], result_mean['NPV'],
    result_std['NPV'], result_mean['test_consum'], result_std['test_consum']]
    temp_df = pd.DataFrame(temp_df)
    temp_df = temp_df.T
    temp_df.columns = df.columns
    df = pd.concat([df, temp_df])


  
time_end = time.time()
print('running time:', time_end - time_start, 's')

running time: 153.8344419002533 s


In [8]:
# Show the result
df

,Infection_rate,Acc,Acc_SD,Sens,Sens_SD,Spec,Spec_SD,PPV,PPV_SD,NPV,NPV_SD,Test_consum,Test_consum_SD
0,0.001,0.989861,0.000327,0.852501,0.035635,0.990001,0.000326,0.079851,0.008803,0.999849,0.000039,100000.0,0.0
0,0.010,0.988598,0.000316,0.849064,0.010490,0.990009,0.000300,0.462084,0.010728,0.998461,0.000114,100000.0,0.0
0,0.030,0.985887,0.000393,0.849974,0.006703,0.990073,0.000319,0.725062,0.007459,0.995355,0.000219,100000.0,0.0
0,0.050,0.983061,0.000396,0.850930,0.005089,0.990012,0.000326,0.817558,0.005445,0.992142,0.000300,100000.0,0.0
0,0.100,0.975959,0.000442,0.849704,0.003048,0.989990,0.000335,0.904148,0.003147,0.983409,0.000367,100000.0,0.0


For table 7 (b)

In [9]:
# table 7 (b)
time_start = time.time()
np.random.seed(0)
df = pd.DataFrame([], columns = ['Infection_rate', 'Sequential_test', 'Stop_rule', 'Repeat', 'Prob_threshold', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD'])
for prob in [0.001, 0.01, 0.03, 0.05, 0.10]:
    temp_data = [fbtk.data_gen(100000, prob) for _ in range(100)]
    for i in [True]:
        for j in [1]:
            for k in [1]:
                
                kwargs = {'stop_rule': j, 'p': prob, 'batch_size': 10,
                'typeII_error': 0.15, 'typeI_error': 0.01, 'repeat': k,
                'prob_threshold': 0.3, 'seq': i}
                test_1 = fbtk.test_result(temp_data, fbtk.seq_test, **kwargs)
                temp_mean = test_1.mean()
                temp_std = test_1.std()
                temp = [kwargs['p'], kwargs['seq'], kwargs['stop_rule'], kwargs['repeat'], kwargs['prob_threshold'],temp_mean['acc'], temp_std['acc'], temp_mean['sens'], temp_std['sens'], temp_mean['spec'], temp_std['spec'], temp_mean['PPV'], temp_std['PPV'], temp_mean['NPV'], temp_std['NPV'], temp_mean['test_consum'], temp_std['test_consum'], temp_mean['ind_consum'], temp_std['ind_consum'], temp_mean['batch_consum'], temp_std['batch_consum']]
                temp_df = pd.DataFrame(temp)
                temp_df = temp_df.T
                temp_df.columns = ['Infection_rate', 'Sequential_test', 'Stop_rule', 'Repeat','Prob_threshold', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
    'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD']
                df = pd.concat([df, temp_df])
            
time_end = time.time()
print('running time:', time_end - time_start, 's')

running time: 117.52977275848389 s


In [10]:
df.to_csv('table7_b.csv')

For table 7 (c)

In [11]:
time_start = time.time()
np.random.seed(0)
df = pd.DataFrame([], columns = ['Infection_rate', 'Sequential_test', 'Stop_rule', 'Repeat', 'Prob_threshold', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD'])
for prob in [0.001, 0.01, 0.03, 0.05, 0.10]:
    temp_data = [fbtk.data_gen(100000, prob) for _ in range(100)]
    for i in [True]:
        for j in [1]:
            for k in [3]:
                batch_size = fbtk.one_batch_test_int_solver(prob, 0.15, 0.01)
                kwargs = {'stop_rule': j, 'p': prob, 'batch_size': batch_size,
                'typeII_error': 0.15, 'typeI_error': 0.01, 'repeat': k,
                'prob_threshold': 0.3, 'seq': i}
                test_1 = fbtk.test_result(temp_data, fbtk.seq_test, **kwargs)
                temp_mean = test_1.mean()
                temp_std = test_1.std()
                temp = [kwargs['p'], kwargs['seq'], kwargs['stop_rule'], kwargs['repeat'], kwargs['prob_threshold'],temp_mean['acc'], temp_std['acc'], temp_mean['sens'], temp_std['sens'], temp_mean['spec'], temp_std['spec'], temp_mean['PPV'], temp_std['PPV'], temp_mean['NPV'], temp_std['NPV'], temp_mean['test_consum'], temp_std['test_consum'], temp_mean['ind_consum'], temp_std['ind_consum'], temp_mean['batch_consum'], temp_std['batch_consum']]
                temp_df = pd.DataFrame(temp)
                temp_df = temp_df.T
                temp_df.columns = ['Infection_rate', 'Sequential_test', 'Stop_rule', 'Repeat','Prob_threshold', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
    'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD']
                df = pd.concat([df, temp_df])
            
time_end = time.time()
print('running time:', time_end - time_start, 's')

running time: 132.6392548084259 s


In [12]:
# table 7 d
time_start = time.time()
np.random.seed(0)
df = pd.DataFrame([], columns = ['Infection_rate', 'Sq_Repeat', 'Ind_Repeat', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD'])
for prob in [0.001, 0.01, 0.03, 0.05, 0.10]:
    temp_data = [fbtk.data_gen(100000, prob) for _ in range(100)]
    for i in [3]: # sq_repeat
        for j in [3]: # ind_repeat
            kwargs = {
                'side_length': 12,
                'typeII_error': 0.15,
                'typeI_error': 0.01,
                'sq_repeat': i,
                'ind_repeat': j
            }
            test_1 = fbtk.test_result(temp_data, fbtk.matrix_test, **kwargs)
            temp_mean = test_1.mean()
            temp_std = test_1.std()
            temp = [prob, kwargs['sq_repeat'], kwargs['ind_repeat'], temp_mean['acc'], temp_std['acc'], temp_mean['sens'], temp_std['sens'], temp_mean['spec'], temp_std['spec'], temp_mean['PPV'], temp_std['PPV'], temp_mean['NPV'], temp_std['NPV'], temp_mean['test_consum'], temp_std['test_consum'], temp_mean['ind_consum'], temp_std['ind_consum'], temp_mean['batch_consum'], temp_std['batch_consum']]
            temp_df = pd.DataFrame(temp)
            temp_df = temp_df.T
            temp_df.columns = ['Infection_rate', 'Sq_Repeat', 'Ind_Repeat', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD']
            df = pd.concat([df, temp_df])

            
                
               
            
time_end = time.time()
print('running time:', time_end - time_start, 's')

running time: 152.28528094291687 s


table 7 (E)

In [13]:
time_start = time.time()
np.random.seed(0)
df = pd.DataFrame([], columns = ['Infection_rate', 'Sequential_test', 'Stop_rule', 'Repeat', 'Prob_threshold', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD'])
for prob in [0.001, 0.01, 0.03, 0.05, 0.10]:
    temp_data = [fbtk.data_gen(100000, prob) for _ in range(100)]
    for i in [True]:
        for j in [3]: # stop_rule
            for k in [1]: # repeat
                batch_size = fbtk.one_batch_test_int_solver(prob, 0.15, 0.01)
                kwargs = {'stop_rule': j, 'p': prob, 'batch_size': batch_size,
                'typeII_error': 0.15, 'typeI_error': 0.01, 'repeat': k,
                'prob_threshold': 0.3, 'seq': i}
                test_1 = fbtk.test_result(temp_data, fbtk.seq_test, **kwargs)
                temp_mean = test_1.mean()
                temp_std = test_1.std()
                temp = [kwargs['p'], kwargs['seq'], kwargs['stop_rule'], kwargs['repeat'], kwargs['prob_threshold'],temp_mean['acc'], temp_std['acc'], temp_mean['sens'], temp_std['sens'], temp_mean['spec'], temp_std['spec'], temp_mean['PPV'], temp_std['PPV'], temp_mean['NPV'], temp_std['NPV'], temp_mean['test_consum'], temp_std['test_consum'], temp_mean['ind_consum'], temp_std['ind_consum'], temp_mean['batch_consum'], temp_std['batch_consum']]
                temp_df = pd.DataFrame(temp)
                temp_df = temp_df.T
                temp_df.columns = ['Infection_rate', 'Sequential_test', 'Stop_rule', 'Repeat','Prob_threshold', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
    'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD']
                df = pd.concat([df, temp_df])
            
time_end = time.time()
print('running time:', time_end - time_start, 's')

running time: 161.998272895813 s


In [210]:
df.to_csv('table7_e.csv')

table 7 (f)

In [14]:
time_start = time.time()
np.random.seed(0)
df = pd.DataFrame([], columns = ['Infection_rate', 'Sequential_test', 'Stop_rule', 'Repeat', 'Prob_threshold', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD'])
for prob in [0.001, 0.01, 0.03, 0.05, 0.10]:
    temp_data = [fbtk.data_gen(100000, prob) for _ in range(100)]
    for i in [True]:
        for j in [3]: # stop_rule
            for k in [3]: # repeat
                batch_size = fbtk.one_batch_test_int_solver(prob, 0.15, 0.01)
                kwargs = {'stop_rule': j, 'p': prob, 'batch_size': batch_size,
                'typeII_error': 0.15, 'typeI_error': 0.01, 'repeat': k,
                'prob_threshold': 0.3, 'seq': i}
                test_1 = fbtk.test_result(temp_data, fbtk.seq_test, **kwargs)
                temp_mean = test_1.mean()
                temp_std = test_1.std()
                temp = [kwargs['p'], kwargs['seq'], kwargs['stop_rule'], kwargs['repeat'], kwargs['prob_threshold'],temp_mean['acc'], temp_std['acc'], temp_mean['sens'], temp_std['sens'], temp_mean['spec'], temp_std['spec'], temp_mean['PPV'], temp_std['PPV'], temp_mean['NPV'], temp_std['NPV'], temp_mean['test_consum'], temp_std['test_consum'], temp_mean['ind_consum'], temp_std['ind_consum'], temp_mean['batch_consum'], temp_std['batch_consum']]
                temp_df = pd.DataFrame(temp)
                temp_df = temp_df.T
                temp_df.columns = ['Infection_rate', 'Sequential_test', 'Stop_rule', 'Repeat','Prob_threshold', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
    'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD']
                df = pd.concat([df, temp_df])
            
time_end = time.time()
print('running time:', time_end - time_start, 's')

running time: 168.818204164505 s


In [213]:
df.to_csv('table7_f.csv')

In [15]:
# appendix A
# table 7 (a)
time_start = time.time()
np.random.seed(0)
df = pd.DataFrame([], columns = ['Infection_rate', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD'])
for prob in [0.001, 0.01, 0.03, 0.05, 0.10]:
    temp_data = [fbtk.data_gen(100000, prob) for _ in range(100)]
    length = len(temp_data)
    acc = np.zeros(length)
    sens = np.zeros(length)
    spec = np.zeros(length)
    ppv = np.zeros(length)
    npv = np.zeros(length)
    test_consum = np.zeros(length)
    for i in range(length):
        pred, consum = fbtk.conventional_test(temp_data[i], typeII_error= 0.25, typeI_error=0.03)
        acc[i] = np.mean(pred[:,1] == temp_data[i][:, 1])
        sens[i] = recall_score(temp_data[i][:, 1], pred[:, 1])
        spec[i] = fbtk.specificity_score(temp_data[i][:, 1], pred[:, 1])
        ppv[i] = precision_score(temp_data[i][:, 1], pred[:, 1])
        npv[i] = fbtk.npv_score(temp_data[i][:, 1], pred[:, 1])
        test_consum[i] = consum
    result = {
        'acc': acc,
        'sens': sens,
        'spec': spec,
        'PPV': ppv,
        'NPV': npv,
        'test_consum': test_consum
    
    }
    result = pd.DataFrame(result)
    result_mean = result.mean()
    result_std = result.std()
    temp_df = [prob, result_mean['acc'], result_std['acc'], result_mean['sens'], result_std['sens'],
    result_mean['spec'], result_std['spec'], result_mean['PPV'], result_std['PPV'], result_mean['NPV'],
    result_std['NPV'], result_mean['test_consum'], result_std['test_consum']]
    temp_df = pd.DataFrame(temp_df)
    temp_df = temp_df.T
    temp_df.columns = df.columns
    df = pd.concat([df, temp_df])


  
time_end = time.time()
print('running time:', time_end - time_start, 's')

running time: 152.71437311172485 s


In [238]:
df.to_csv('appendix_a.csv')

In [16]:
# Appendix (b)
time_start = time.time()
np.random.seed(0)
df = pd.DataFrame([], columns = ['Infection_rate', 'Sequential_test', 'Stop_rule', 'Repeat', 'Prob_threshold', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD'])
for prob in [0.001, 0.01, 0.03, 0.05, 0.10]:
    temp_data = [fbtk.data_gen(100000, prob) for _ in range(100)]
    for i in [True]:
        for j in [1]:
            for k in [1]:
                
                kwargs = {'stop_rule': j, 'p': prob, 'batch_size': 10,
                'typeII_error': 0.25, 'typeI_error': 0.03, 'repeat': k,
                'prob_threshold': 0.3, 'seq': i}
                test_1 = fbtk.test_result(temp_data, fbtk.seq_test, **kwargs)
                temp_mean = test_1.mean()
                temp_std = test_1.std()
                temp = [kwargs['p'], kwargs['seq'], kwargs['stop_rule'], kwargs['repeat'], kwargs['prob_threshold'],temp_mean['acc'], temp_std['acc'], temp_mean['sens'], temp_std['sens'], temp_mean['spec'], temp_std['spec'], temp_mean['PPV'], temp_std['PPV'], temp_mean['NPV'], temp_std['NPV'], temp_mean['test_consum'], temp_std['test_consum'], temp_mean['ind_consum'], temp_std['ind_consum'], temp_mean['batch_consum'], temp_std['batch_consum']]
                temp_df = pd.DataFrame(temp)
                temp_df = temp_df.T
                temp_df.columns = ['Infection_rate', 'Sequential_test', 'Stop_rule', 'Repeat','Prob_threshold', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
    'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD']
                df = pd.concat([df, temp_df])
            
time_end = time.time()
print('running time:', time_end - time_start, 's')

running time: 117.7569146156311 s


In [242]:
df.to_csv('appendix_b.csv')

In [17]:
# Appendix (c)
time_start = time.time()
np.random.seed(0)
df = pd.DataFrame([], columns = ['Infection_rate', 'Sequential_test', 'Stop_rule', 'Repeat', 'Prob_threshold', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD'])
for prob in [0.001, 0.01, 0.03, 0.05, 0.10]:
    temp_data = [fbtk.data_gen(100000, prob) for _ in range(100)]
    for i in [True]:
        for j in [1]:
            for k in [3]:
                batch_size = fbtk.one_batch_test_int_solver(prob, 0.25, 0.03)
                kwargs = {'stop_rule': j, 'p': prob, 'batch_size': batch_size,
                'typeII_error': 0.25, 'typeI_error': 0.03, 'repeat': k,
                'prob_threshold': 0.3, 'seq': i}
                test_1 = fbtk.test_result(temp_data, fbtk.seq_test, **kwargs)
                temp_mean = test_1.mean()
                temp_std = test_1.std()
                temp = [kwargs['p'], kwargs['seq'], kwargs['stop_rule'], kwargs['repeat'], kwargs['prob_threshold'],temp_mean['acc'], temp_std['acc'], temp_mean['sens'], temp_std['sens'], temp_mean['spec'], temp_std['spec'], temp_mean['PPV'], temp_std['PPV'], temp_mean['NPV'], temp_std['NPV'], temp_mean['test_consum'], temp_std['test_consum'], temp_mean['ind_consum'], temp_std['ind_consum'], temp_mean['batch_consum'], temp_std['batch_consum']]
                temp_df = pd.DataFrame(temp)
                temp_df = temp_df.T
                temp_df.columns = ['Infection_rate', 'Sequential_test', 'Stop_rule', 'Repeat','Prob_threshold', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
    'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD']
                df = pd.concat([df, temp_df])
            
time_end = time.time()
print('running time:', time_end - time_start, 's')

running time: 131.44559955596924 s


In [247]:
df.to_csv('appendix_c.csv')

In [18]:
# Appendix (d)
time_start = time.time()
np.random.seed(0)
df = pd.DataFrame([], columns = ['Infection_rate', 'Sq_Repeat', 'Ind_Repeat', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD'])
for prob in [0.001, 0.01, 0.03, 0.05, 0.10]:
    temp_data = [fbtk.data_gen(100000, prob) for _ in range(100)]
    for i in [3]: # sq_repeat
        for j in [3]: # ind_repeat
            kwargs = {
                'side_length': 12,
                'typeII_error': 0.25,
                'typeI_error': 0.03,
                'sq_repeat': i,
                'ind_repeat': j
            }
            test_1 = fbtk.test_result(temp_data, fbtk.matrix_test, **kwargs)
            temp_mean = test_1.mean()
            temp_std = test_1.std()
            temp = [prob, kwargs['sq_repeat'], kwargs['ind_repeat'], temp_mean['acc'], temp_std['acc'], temp_mean['sens'], temp_std['sens'], temp_mean['spec'], temp_std['spec'], temp_mean['PPV'], temp_std['PPV'], temp_mean['NPV'], temp_std['NPV'], temp_mean['test_consum'], temp_std['test_consum'], temp_mean['ind_consum'], temp_std['ind_consum'], temp_mean['batch_consum'], temp_std['batch_consum']]
            temp_df = pd.DataFrame(temp)
            temp_df = temp_df.T
            temp_df.columns = ['Infection_rate', 'Sq_Repeat', 'Ind_Repeat', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD']
            df = pd.concat([df, temp_df])

            
                
               
            
time_end = time.time()
print('running time:', time_end - time_start, 's')

running time: 154.8913700580597 s


In [250]:
df.to_csv('appendix_d.csv')

In [19]:
# Appendix e
time_start = time.time()
np.random.seed(0)
df = pd.DataFrame([], columns = ['Infection_rate', 'Sequential_test', 'Stop_rule', 'Repeat', 'Prob_threshold', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD'])
for prob in [0.001, 0.01, 0.03, 0.05, 0.10]:
    temp_data = [fbtk.data_gen(100000, prob) for _ in range(100)]
    for i in [True]:
        for j in [3]: # stop_rule
            for k in [1]: # repeat
                batch_size = fbtk.one_batch_test_int_solver(prob, 0.25, 0.03)
                kwargs = {'stop_rule': j, 'p': prob, 'batch_size': batch_size,
                'typeII_error': 0.25, 'typeI_error': 0.03, 'repeat': k,
                'prob_threshold': 0.3, 'seq': i}
                test_1 = fbtk.test_result(temp_data, fbtk.seq_test, **kwargs)
                temp_mean = test_1.mean()
                temp_std = test_1.std()
                temp = [kwargs['p'], kwargs['seq'], kwargs['stop_rule'], kwargs['repeat'], kwargs['prob_threshold'],temp_mean['acc'], temp_std['acc'], temp_mean['sens'], temp_std['sens'], temp_mean['spec'], temp_std['spec'], temp_mean['PPV'], temp_std['PPV'], temp_mean['NPV'], temp_std['NPV'], temp_mean['test_consum'], temp_std['test_consum'], temp_mean['ind_consum'], temp_std['ind_consum'], temp_mean['batch_consum'], temp_std['batch_consum']]
                temp_df = pd.DataFrame(temp)
                temp_df = temp_df.T
                temp_df.columns = ['Infection_rate', 'Sequential_test', 'Stop_rule', 'Repeat','Prob_threshold', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
    'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD']
                df = pd.concat([df, temp_df])
            
time_end = time.time()
print('running time:', time_end - time_start, 's')

running time: 165.98010206222534 s


In [254]:
# Appendix f
time_start = time.time()
np.random.seed(0)
df = pd.DataFrame([], columns = ['Infection_rate', 'Sequential_test', 'Stop_rule', 'Repeat', 'Prob_threshold', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD'])
for prob in [0.001, 0.01, 0.03, 0.05, 0.10]:
    temp_data = [fbtk.data_gen(100000, prob) for _ in range(100)]
    for i in [True]:
        for j in [3]: # stop_rule
            for k in [3]: # repeat
                batch_size = fbtk.one_batch_test_int_solver(prob, 0.25, 0.03)
                kwargs = {'stop_rule': j, 'p': prob, 'batch_size': batch_size,
                'typeII_error': 0.25, 'typeI_error': 0.03, 'repeat': k,
                'prob_threshold': 0.3, 'seq': i}
                test_1 = fbtk.test_result(temp_data, fbtk.seq_test, **kwargs)
                temp_mean = test_1.mean()
                temp_std = test_1.std()
                temp = [kwargs['p'], kwargs['seq'], kwargs['stop_rule'], kwargs['repeat'], kwargs['prob_threshold'],temp_mean['acc'], temp_std['acc'], temp_mean['sens'], temp_std['sens'], temp_mean['spec'], temp_std['spec'], temp_mean['PPV'], temp_std['PPV'], temp_mean['NPV'], temp_std['NPV'], temp_mean['test_consum'], temp_std['test_consum'], temp_mean['ind_consum'], temp_std['ind_consum'], temp_mean['batch_consum'], temp_std['batch_consum']]
                temp_df = pd.DataFrame(temp)
                temp_df = temp_df.T
                temp_df.columns = ['Infection_rate', 'Sequential_test', 'Stop_rule', 'Repeat','Prob_threshold', 'Acc', 'Acc_SD', 'Sens', 'Sens_SD', 'Spec','Spec_SD','PPV', 'PPV_SD',
    'NPV', 'NPV_SD', 'Test_consum', 'Test_consum_SD', 'Ind_consum', 'Ind_consum_SD', 'Batch_consum','Batch_consum_SD']
                df = pd.concat([df, temp_df])
            
time_end = time.time()
print('running time:', time_end - time_start, 's')

time cost: 177.40325736999512 s
